# Clone Forge

In [ ]:
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git

# Install Requirements

In [ ]:
%cd /kaggle/working/stable-diffusion-webui-forge
!pip install -r /kaggle/working/stable-diffusion-webui-forge/requirements_versions.txt

# Run

#### *Note!*

*Update with your ngrok token*

In [ ]:
# --- Variables ---

Ngrok_token = ""  # Update with your ngrok token
Ngrok_domain = ""  # optional, leave empty if you don't have a domain
port = 7860
HOME_FOLDER = "/kaggle"

# -----------------

!pip install pyngrok==6.1.0  # Downgrade to a version that supports ngrok tunneling

from pyngrok import ngrok, conf
import gc

gc.collect()

if Ngrok_token:
    try:
        ngrok.set_auth_token(Ngrok_token)
        ngrok.kill()
        srv = ngrok.connect(port)
        print(f"Ngrok Tunnel is active at: {srv.public_url}")
        
        # Command to start
        !echo -e "A\nN" | python /kaggle/working/stable-diffusion-webui-forge/launch.py
        
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
else:
    print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the Ngrok_token field.')

# Install Model

### Install Checkpoints

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
# You can find other model and download it here just chage name and model url
model_url = "https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors?download=true"
model_name = "flux1-dev-bnb-nf4-v2.safetensors"

%cd /kaggle/working/stable-diffusion-webui-forge/models/Stable-diffusion
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### Install *Lora*



In [ ]:

model_url = "https://civitai.com/api/download/models/114163"
model_name = "Jellyfishforest.safetensors"


%cd /kaggle/working/stable-diffusion-webui-forge/models/Lora
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Launch with Remote Moe

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe
port = 7860

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python /kaggle/working/stable-diffusion-webui-forge/launch.py & ssh -R 80:127.0.0.1:$port -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Launch with Piggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/stable-diffusion-webui-forge/launch.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7860 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()